In [1]:
import numpy as np
import pandas as pd
import re
import datetime

In [2]:
index = pd.read_csv('data/index_data_total.csv')

In [3]:
index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53323 entries, 0 to 53322
Data columns (total 9 columns):
date          53323 non-null object
time          37905 non-null object
currency      53323 non-null object
impact        53323 non-null object
index_name    53323 non-null object
actual        43191 non-null object
actual_stu    53323 non-null int64
forecast      34913 non-null object
previous      43155 non-null object
dtypes: int64(1), object(8)
memory usage: 3.7+ MB


In [4]:
index.head()

,date,time,currency,impact,index_name,actual,actual_stu,forecast,previous
0,MonJan 1,All Day,NZD,holiday,Bank Holiday,NaN,0,NaN,NaN
1,,All Day,AUD,holiday,Bank Holiday,NaN,0,NaN,NaN
2,,All Day,CNY,holiday,Bank Holiday,NaN,0,NaN,NaN
3,,All Day,JPY,holiday,Bank Holiday,NaN,0,NaN,NaN
4,,10:00am,CNY,high,Manufacturing PMI,54.8,0,NaN,55.3


### 시간 관련 변수

#### 연도

In [5]:
# 연도 파생변수 생성
years = np.arange(2007, 2019).tolist()

for year in years:
    index.loc[index['date'].str.contains('.*Jan 1 $'), 'year'] = years
index.loc[index['date'].str.contains('SunJan 2 $'), 'year'] = 2011

index['year'] = index['year'].fillna(method='ffill')

index['year'] = index['year'].astype(int)
index['year'] = index['year'].astype(str)

#### 월/일/요일

In [6]:
# 대문자를 기준으로 문자열 나누기
def string_split(string) :    
    answer = string[0]
    for idx, char in enumerate(string[1:]):
        if string[idx].islower() and char.isupper():
            answer += ' '
        answer += char
    return answer

In [7]:
# date 변수 정리
index['date'] = index['date'].apply(lambda x : string_split(x))
index['date'] = index['date'].apply(lambda x : x[1:-1])

index.loc[index['date']=='', 'date']=np.nan

index['date'] = index['date'].fillna(method='ffill')

#### 시간

In [8]:
pd.Series.value_counts(index['time'].str.contains(':') == False)

False    50802
True      2521
Name: time, dtype: int64

In [9]:
# time변수 중 시간형태가 아닌 종류
index.loc[index['time'].str.contains(':') == False, 'time'].value_counts()

All Day     2123
Day 2        152
Day 1        152
Day 3         47
Day 4         14
Jul Data      10
Day 5          7
Sep Data       5
Jan Data       3
Aug Data       1
Jun Data       1
Mar Data       1
Oct Data       1
Feb Data       1
Dec Data       1
Q2 Data        1
Nov Data       1
Name: time, dtype: int64

In [10]:
# time 변수 중 시간형태 아닌 변수들 파생변수 생성

index['time'] = index['time'].fillna(method='ffill')

index.loc[index['time'].str.contains('All Day'), 'time_Allday'] = 1
index['time_Allday'].fillna(0, inplace = True)

index.loc[index['time'].str.contains('Day 1'), 'time_Event'] = 1
index.loc[index['time'].str.contains('Day 2'), 'time_Event'] = 2
index.loc[index['time'].str.contains('Day 3'), 'time_Event'] = 3
index.loc[index['time'].str.contains('Day 4'), 'time_Event'] = 4
index.loc[index['time'].str.contains('Day 5'), 'time_Event'] = 5
index['time_Event'].fillna(0, inplace = True)

index.loc[index['time'].str.contains('Jan Data'), 'time_month'] = 1
index.loc[index['time'].str.contains('Feb Data'), 'time_month'] = 2
index.loc[index['time'].str.contains('Mar Data'), 'time_month'] = 3
index.loc[index['time'].str.contains('Apr Data'), 'time_month'] = 4
index.loc[index['time'].str.contains('May Data'), 'time_month'] = 5
index.loc[index['time'].str.contains('Jun Data'), 'time_month'] = 6
index.loc[index['time'].str.contains('Jul Data'), 'time_month'] = 7
index.loc[index['time'].str.contains('Aug Data'), 'time_month'] = 8
index.loc[index['time'].str.contains('Sep Data'), 'time_month'] = 9
index.loc[index['time'].str.contains('Oct Data'), 'time_month'] = 10
index.loc[index['time'].str.contains('Nov Data'), 'time_month'] = 11
index.loc[index['time'].str.contains('Dec Data'), 'time_month'] = 12
index['time_month'].fillna(0, inplace = True)

index.loc[index['time'].str.contains('Q1 Data'), 'time_Quarter'] = 1
index.loc[index['time'].str.contains('Q2 Data'), 'time_Quarter'] = 2
index.loc[index['time'].str.contains('Q3 Data'), 'time_Quarter'] = 3
index['time_Quarter'].fillna(0, inplace = True)

In [11]:
# time 변수 정리(시간형태는 시간데이터로, 그 외의 형태는 00:00로 변형)

index.loc[index['time'].apply(lambda x : len(x)) == 6, 'time'] = '0' + index['time']
index['time'] = index['time'].str.upper()

index.loc[index['time'].str.contains(':') == False, 'time'] = '12:00AM'

#### datetime 타입으로 변형

In [12]:
# datetime 변수 생성
index['date'] = index['date'] + ' ' + index['year']
index.loc[index['time'].str.contains(':'), 'datetime'] = index['date'] + ' ' + index['time'] # time에서 시간형태인 데이터만
index['datetime'] = index.loc[index['datetime'].str.contains(':')==True, 'datetime'].apply(lambda x : datetime.datetime.strptime(x, "%a %b %d %Y %I:%M%p"))

In [13]:
# 월, 일, 요일 파생변수 생성
index['month'] = index['datetime'].apply(lambda x : x.month)
index['day'] = index['datetime'].apply(lambda x : x.day)
index['weekday'] = index['datetime'].apply(lambda x : x.weekday())

In [14]:
index.head()

,date,time,currency,impact,index_name,actual,actual_stu,forecast,previous,year,time_Allday,time_Event,time_month,time_Quarter,datetime,month,day,weekday
0,Mon Jan 1 2007,12:00AM,NZD,holiday,Bank Holiday,NaN,0,NaN,NaN,2007,1.0,0.0,0.0,0.0,2007-01-01 00:00:00,1,1,0
1,Mon Jan 1 2007,12:00AM,AUD,holiday,Bank Holiday,NaN,0,NaN,NaN,2007,1.0,0.0,0.0,0.0,2007-01-01 00:00:00,1,1,0
2,Mon Jan 1 2007,12:00AM,CNY,holiday,Bank Holiday,NaN,0,NaN,NaN,2007,1.0,0.0,0.0,0.0,2007-01-01 00:00:00,1,1,0
3,Mon Jan 1 2007,12:00AM,JPY,holiday,Bank Holiday,NaN,0,NaN,NaN,2007,1.0,0.0,0.0,0.0,2007-01-01 00:00:00,1,1,0
4,Mon Jan 1 2007,10:00AM,CNY,high,Manufacturing PMI,54.8,0,NaN,55.3,2007,0.0,0.0,0.0,0.0,2007-01-01 10:00:00,1,1,0


### 수치 관련 변수

In [15]:
# 수치 관련 데이터들 분석을 위해 결측치 확인
index.isna().sum()

date                0
time                0
currency            0
impact              0
index_name          0
actual          10132
actual_stu          0
forecast        18410
previous        10168
year                0
time_Allday         0
time_Event          0
time_month          0
time_Quarter        0
datetime            0
month               0
day                 0
weekday             0
dtype: int64

* 수치 관련 데이터들만 NA값 존재
* 예측값의 결측치가 가장 많으며, 실제값의 결측치가 가장 적은 것을 확인할 수 있다.

In [16]:
index.fillna('NA', inplace = True) # 결측치는 일단 'NA'로 통일

#### 실제치(actual)

In [17]:
# actual 변수정리 및 파생변수 생성

index.loc[index['actual'].str.contains('<'), 'actual'] = index.loc[index['actual'].str.contains('<'), 'actual'].apply(lambda x : x[1:])
index.loc[index['actual'].str.contains('%'), 'actual'] = index.loc[index['actual'].str.contains('%'), 'actual'].apply(lambda x : x[:-1]).astype(float).apply(lambda x : x/100).astype(str)
index.loc[index['actual'].str.contains('B'), 'actual'] = index.loc[index['actual'].str.contains('B'), 'actual'].apply(lambda x : x[:-1]).astype(float).apply(lambda x : x*1000000000).astype(int).astype(str)
index.loc[index['actual'].str.contains('K'), 'actual'] = index.loc[index['actual'].str.contains('K'), 'actual'].apply(lambda x : x[:-1]).astype(float).apply(lambda x : x*1000).astype(int).astype(str)
index.loc[index['actual'].str.contains('M'), 'actual'] = index.loc[index['actual'].str.contains('M'), 'actual'].apply(lambda x : x[:-1]).astype(float).apply(lambda x : x*1000000).astype(int).astype(str)
index.loc[index['actual'].str.contains('T'), 'actual'] = index.loc[index['actual'].str.contains('T'), 'actual'].apply(lambda x : x[:-1]).astype(float).apply(lambda x : x*1000000000000).astype(int).astype(str)

index['Status'] = index.loc[index['actual'].str.contains('[a-zA-Z]'), 'actual'] # index가 아닌 사건의 상태(Yes, No, Pass, Reject, Leave)
index['Status'].fillna('NA', inplace = True)
index.loc[index['Status'].str.contains('Yes'), 'Status_Yes'] = 1
index['Status_Yes'].fillna(0, inplace = True)
index.loc[index['Status'].str.contains('No'), 'Status_No'] = 1
index['Status_No'].fillna(0, inplace = True)
index.loc[index['Status'].str.contains('Pass'), 'Status_Pass'] = 1
index['Status_Pass'].fillna(0, inplace = True)
index.loc[index['Status'].str.contains('Reject'), 'Status_Reject'] = 1
index['Status_Reject'].fillna(0, inplace = True)
index.loc[index['Status'].str.contains('Leave'), 'Status_Leave'] = 1
index['Status_Leave'].fillna(0, inplace = True)

index['bond_actual'] = index.loc[index['actual'].str.contains('[|]'), 'actual'] # 채권 이율/bid
index['bond_actual'].fillna('NA', inplace = True)
index['bond_a_interest'] = index.loc[index['bond_actual'].str.contains('[|]'), 'bond_actual'].apply(lambda x : x.split('|')).apply(lambda x : x[0])
index['bond_a_interest'].fillna(0, inplace = True)
index['bond_a_bid'] = index.loc[index['bond_actual'].str.contains('[|]'), 'bond_actual'].apply(lambda x : x.split('|')).apply(lambda x : x[1])
index['bond_a_bid'].fillna(0, inplace = True)

index['Votes'] = index.loc[index['actual'].str.contains('\d-\d-\d'), 'actual'] # 투표 찬성/반대/기권
index['Votes'].fillna('NA', inplace = True)
index['Votes_support'] = index.loc[index['Votes'].str.contains('\d-\d-\d'), 'Votes'].apply(lambda x : x.split('-')).apply(lambda x : x[0])
index['Votes_support'].fillna(0, inplace = True)
index['Votes_oppose'] = index.loc[index['Votes'].str.contains('\d-\d-\d'), 'Votes'].apply(lambda x : x.split('-')).apply(lambda x : x[1])
index['Votes_oppose'].fillna(0, inplace = True)
index['Votes_neutral'] = index.loc[index['Votes'].str.contains('\d-\d-\d'), 'Votes'].apply(lambda x : x.split('-')).apply(lambda x : x[2])
index['Votes_neutral'].fillna(0, inplace = True)

index['num_actual'] = index['actual']
index.loc[index['num_actual'].str.contains('[a-zA-Z]'), 'num_actual'] = '0'
index.loc[index['num_actual'].str.contains('[|]'), 'num_actual'] = '0'
index.loc[index['num_actual'].str.contains('\d-\d-\d'), 'num_actual'] = '0'
index.loc[index['num_actual'].str.contains('NA'), 'num_actual'] = '0'

index['num_actual'] = index['num_actual'].astype(float)

#### 예측치(forecast)

In [18]:
# forecast 변수정리 및 파생변수 생성

index.loc[index['forecast'].str.contains('<'), 'forecast'] = index.loc[index['forecast'].str.contains('<'), 'forecast'].apply(lambda x : x[1:])
index.loc[index['forecast'].str.contains('%'), 'forecast'] = index.loc[index['forecast'].str.contains('%'), 'forecast'].apply(lambda x : x[:-2] if '%%' in x else x[:-1]).astype(float).apply(lambda x : x/100).astype(str)
index.loc[index['forecast'].str.contains('B'), 'forecast'] = index.loc[index['forecast'].str.contains('B'), 'forecast'].apply(lambda x : x[:-1]).astype(float).apply(lambda x : x*1000000000).astype(int).astype(str)
index.loc[index['forecast'].str.contains('K'), 'forecast'] = index.loc[index['forecast'].str.contains('K'), 'forecast'].apply(lambda x : x[:-1]).astype(float).apply(lambda x : x*1000).astype(int).astype(str)
index.loc[index['forecast'].str.contains('M'), 'forecast'] = index.loc[index['forecast'].str.contains('M'), 'forecast'].apply(lambda x : x[:-1]).astype(float).apply(lambda x : x*1000000).astype(int).astype(str)
index.loc[index['forecast'].str.contains('T'), 'forecast'] = index.loc[index['forecast'].str.contains('T'), 'forecast'].apply(lambda x : x[:-1]).astype(float).apply(lambda x : x*1000000000000).astype(int).astype(str)

index['f_Status'] = index.loc[index['forecast'].str.contains('[a-zA-Z]'), 'forecast'] # index가 아닌 사건의 상태(Yes, No, Pass, Reject, Leave)
index['f_Status'].fillna('NA', inplace = True)
index.loc[index['f_Status'].str.contains('Yes'), 'f_Status_Yes'] = 1
index['f_Status_Yes'].fillna(0, inplace = True)
index.loc[index['f_Status'].str.contains('No'), 'f_Status_No'] = 1
index['f_Status_No'].fillna(0, inplace = True)
index.loc[index['f_Status'].str.contains('Pass'), 'f_Status_Pass'] = 1
index['f_Status_Pass'].fillna(0, inplace = True)
index.loc[index['f_Status'].str.contains('Reject'), 'f_Status_Reject'] = 1
index['f_Status_Reject'].fillna(0, inplace = True)
index.loc[index['f_Status'].str.contains('Leave'), 'f_Status_Leave'] = 1
index['f_Status_Leave'].fillna(0, inplace = True)

index['bond_forecast'] = index.loc[index['forecast'].str.contains('[|]'), 'forecast'] # 채권 이율/bid
index['bond_forecast'].fillna('NA', inplace = True)
index['bond_f_interest'] = index.loc[index['bond_forecast'].str.contains('[|]'), 'bond_forecast'].apply(lambda x : x.split('|')).apply(lambda x : x[0])
index['bond_f_interest'].fillna(0, inplace = True)
index['bond_f_bid'] = index.loc[index['bond_forecast'].str.contains('[|]'), 'bond_forecast'].apply(lambda x : x.split('|')).apply(lambda x : x[1])
index['bond_f_bid'].fillna(0, inplace = True)

index['f_Votes'] = index.loc[index['forecast'].str.contains('\d-\d-\d'), 'forecast'] # 투표 찬성/반대/기권
index['f_Votes'].fillna('NA', inplace = True)
index['f_Votes_support'] = index.loc[index['f_Votes'].str.contains('\d-\d-\d'), 'f_Votes'].apply(lambda x : x.split('-')).apply(lambda x : x[0])
index['f_Votes_support'].fillna(0, inplace = True)
index['f_Votes_oppose'] = index.loc[index['f_Votes'].str.contains('\d-\d-\d'), 'f_Votes'].apply(lambda x : x.split('-')).apply(lambda x : x[1])
index['f_Votes_oppose'].fillna(0, inplace = True)
index['f_Votes_neutral'] = index.loc[index['f_Votes'].str.contains('\d-\d-\d'), 'f_Votes'].apply(lambda x : x.split('-')).apply(lambda x : x[2])
index['f_Votes_neutral'].fillna(0, inplace = True)

index['num_forecast'] = index['forecast']
index.loc[index['num_forecast'].str.contains('[a-zA-Z]'), 'num_forecast'] = '0'
index.loc[index['num_forecast'].str.contains('[|]'), 'num_forecast'] = '0'
index.loc[index['num_forecast'].str.contains('\d-\d-\d'), 'num_forecast'] = '0'
index.loc[index['num_forecast'].str.contains('NA'), 'num_forecast'] = '0'

index['num_forecast'] = index['num_forecast'].astype(float)

#### 이전치(previous)

In [19]:
# previous 변수정리 및 파생변수 생성

index.loc[index['previous'].str.contains('<'), 'previous'] = index.loc[index['previous'].str.contains('<'), 'previous'].apply(lambda x : x[1:])
index.loc[index['previous'].str.contains('%'), 'previous'] = index.loc[index['previous'].str.contains('%'), 'previous'].apply(lambda x : x[:-1]).astype(float).apply(lambda x : x/100).astype(str)
index.loc[index['previous'].str.contains('B'), 'previous'] = index.loc[index['previous'].str.contains('B'), 'previous'].apply(lambda x : x[:-1]).astype(float).apply(lambda x : x*1000000000).astype(int).astype(str)
index.loc[index['previous'].str.contains('K'), 'previous'] = index.loc[index['previous'].str.contains('K'), 'previous'].apply(lambda x : x[:-1]).astype(float).apply(lambda x : x*1000).astype(int).astype(str)
index.loc[index['previous'].str.contains('M'), 'previous'] = index.loc[index['previous'].str.contains('M'), 'previous'].apply(lambda x : x[:-1]).astype(float).apply(lambda x : x*1000000).astype(int).astype(str)
index.loc[index['previous'].str.contains('T'), 'previous'] = index.loc[index['previous'].str.contains('T'), 'previous'].apply(lambda x : x[:-1]).astype(float).apply(lambda x : x*1000000000000).astype(int).astype(str)

index['p_Status'] = index.loc[index['previous'].str.contains('[a-zA-Z]'), 'previous'] # index가 아닌 사건의 상태(Yes, No, Pass, Reject, Leave)
index['p_Status'].fillna('NA', inplace = True)
index.loc[index['p_Status'].str.contains('Yes'), 'p_Status_Yes'] = 1
index['p_Status_Yes'].fillna(0, inplace = True)
index.loc[index['p_Status'].str.contains('No'), 'p_Status_No'] = 1
index['p_Status_No'].fillna(0, inplace = True)
index.loc[index['p_Status'].str.contains('Pass'), 'p_Status_Pass'] = 1
index['p_Status_Pass'].fillna(0, inplace = True)
index.loc[index['p_Status'].str.contains('Reject'), 'p_Status_Reject'] = 1
index['p_Status_Reject'].fillna(0, inplace = True)
index.loc[index['p_Status'].str.contains('Leave'), 'p_Status_Leave'] = 1
index['p_Status_Leave'].fillna(0, inplace = True)

index['bond_previous'] = index.loc[index['previous'].str.contains('[|]'), 'previous'] # 채권 이율/bid
index['bond_previous'].fillna('NA', inplace = True)
index['bond_p_interest'] = index.loc[index['bond_previous'].str.contains('[|]'), 'bond_previous'].apply(lambda x : x.split('|')).apply(lambda x : x[0])
index['bond_p_interest'].fillna(0, inplace = True)
index['bond_p_bid'] = index.loc[index['bond_previous'].str.contains('[|]'), 'bond_previous'].apply(lambda x : x.split('|')).apply(lambda x : x[1])
index['bond_p_bid'].fillna(0, inplace = True)

index['p_Votes'] = index.loc[index['previous'].str.contains('\d-\d-\d'), 'previous'] # 투표 찬성/반대/기권
index['p_Votes'].fillna('NA', inplace = True)
index['p_Votes_support'] = index.loc[index['p_Votes'].str.contains('\d-\d-\d'), 'p_Votes'].apply(lambda x : x.split('-')).apply(lambda x : x[0])
index['p_Votes_support'].fillna(0, inplace = True)
index['p_Votes_oppose'] = index.loc[index['p_Votes'].str.contains('\d-\d-\d'), 'p_Votes'].apply(lambda x : x.split('-')).apply(lambda x : x[1])
index['p_Votes_oppose'].fillna(0, inplace = True)
index['p_Votes_neutral'] = index.loc[index['p_Votes'].str.contains('\d-\d-\d'), 'p_Votes'].apply(lambda x : x.split('-')).apply(lambda x : x[2])
index['p_Votes_neutral'].fillna(0, inplace = True)

index['num_previous'] = index['previous']
index.loc[index['num_previous'].str.contains('[a-zA-Z]'), 'num_previous'] = '0'
index.loc[index['num_previous'].str.contains('[|]'), 'num_previous'] = '0'
index.loc[index['num_previous'].str.contains('\d-\d-\d'), 'num_previous'] = '0'
index.loc[index['num_previous'].str.contains('NA'), 'num_previous'] = '0'

index['num_previous'] = index['num_previous'].astype(float)

In [20]:
# 생성 파생변수 중 중복되는 변수 제거
index.drop(['Status', 'bond_actual', 'Votes', 'f_Status', 'bond_forecast', 'f_Votes', 'p_Status', 'bond_previous', 'p_Votes'], axis=1, inplace = True)

In [21]:
index.columns

Index(['date', 'time', 'currency', 'impact', 'index_name', 'actual',
       'actual_stu', 'forecast', 'previous', 'year', 'time_Allday',
       'time_Event', 'time_month', 'time_Quarter', 'datetime', 'month', 'day',
       'weekday', 'Status_Yes', 'Status_No', 'Status_Pass', 'Status_Reject',
       'Status_Leave', 'bond_a_interest', 'bond_a_bid', 'Votes_support',
       'Votes_oppose', 'Votes_neutral', 'num_actual', 'f_Status_Yes',
       'f_Status_No', 'f_Status_Pass', 'f_Status_Reject', 'f_Status_Leave',
       'bond_f_interest', 'bond_f_bid', 'f_Votes_support', 'f_Votes_oppose',
       'f_Votes_neutral', 'num_forecast', 'p_Status_Yes', 'p_Status_No',
       'p_Status_Pass', 'p_Status_Reject', 'p_Status_Leave', 'bond_p_interest',
       'bond_p_bid', 'p_Votes_support', 'p_Votes_oppose', 'p_Votes_neutral',
       'num_previous'],
      dtype='object')

#### 가중치(impact)

In [22]:
index.loc[index['impact']=='holiday', 'impact'] = 0
index.loc[index['impact']=='low', 'impact'] = 1
index.loc[index['impact']=='medium', 'impact'] = 2
index.loc[index['impact']=='high', 'impact'] = 3

In [23]:
index['impact'].value_counts()

1    20206
2    18748
3    12804
0     1565
Name: impact, dtype: int64

#### 통화단위(국가)(currency)

In [24]:
index['currency'].value_counts()

USD    13435
EUR    11423
GBP     7154
JPY     5278
AUD     4560
CAD     3836
NZD     2802
CNY     2320
CHF     2143
All      372
Name: currency, dtype: int64

In [25]:
# 가변수를 생성하여 one-hot encoding
currency_dumm = pd.get_dummies(index['currency'])
index = pd.merge(index, currency_dumm, on=index.index)
index.drop(['key_0'], axis=1, inplace = True)

In [26]:
index.columns

Index(['date', 'time', 'currency', 'impact', 'index_name', 'actual',
       'actual_stu', 'forecast', 'previous', 'year', 'time_Allday',
       'time_Event', 'time_month', 'time_Quarter', 'datetime', 'month', 'day',
       'weekday', 'Status_Yes', 'Status_No', 'Status_Pass', 'Status_Reject',
       'Status_Leave', 'bond_a_interest', 'bond_a_bid', 'Votes_support',
       'Votes_oppose', 'Votes_neutral', 'num_actual', 'f_Status_Yes',
       'f_Status_No', 'f_Status_Pass', 'f_Status_Reject', 'f_Status_Leave',
       'bond_f_interest', 'bond_f_bid', 'f_Votes_support', 'f_Votes_oppose',
       'f_Votes_neutral', 'num_forecast', 'p_Status_Yes', 'p_Status_No',
       'p_Status_Pass', 'p_Status_Reject', 'p_Status_Leave', 'bond_p_interest',
       'bond_p_bid', 'p_Votes_support', 'p_Votes_oppose', 'p_Votes_neutral',
       'num_previous', 'AUD', 'All', 'CAD', 'CHF', 'CNY', 'EUR', 'GBP', 'JPY',
       'NZD', 'USD'],
      dtype='object')

In [27]:
# 파생변수 및 가변수 생성으로 인한 중복변수 제거
index.drop(['date', 'time', 'currency'], axis = 1, inplace = True)

In [28]:
index = index.set_index("datetime")

In [29]:
col_array = pd.Series(list(index.columns.values))

In [30]:
col_array

0              impact
1          index_name
2              actual
3          actual_stu
4            forecast
5            previous
6                year
7         time_Allday
8          time_Event
9          time_month
10       time_Quarter
11              month
12                day
13            weekday
14         Status_Yes
15          Status_No
16        Status_Pass
17      Status_Reject
18       Status_Leave
19    bond_a_interest
20         bond_a_bid
21      Votes_support
22       Votes_oppose
23      Votes_neutral
24         num_actual
25       f_Status_Yes
26        f_Status_No
27      f_Status_Pass
28    f_Status_Reject
29     f_Status_Leave
30    bond_f_interest
31         bond_f_bid
32    f_Votes_support
33     f_Votes_oppose
34    f_Votes_neutral
35       num_forecast
36       p_Status_Yes
37        p_Status_No
38      p_Status_Pass
39    p_Status_Reject
40     p_Status_Leave
41    bond_p_interest
42         bond_p_bid
43    p_Votes_support
44     p_Votes_oppose
45    p_Vo

In [31]:
new_order = [1, 0, 3, 2, 4, 5, 24, 35, 46, 19, 20, 30, 31, 41, 42, 14, 15, 16, 17, 18, 25, 26, 27, 28, 29, 36, 37, 38, 39, 40, 21, 22, 23, 32, 33, 34, 43, 44, 45, 6, 11, 12, 13, 7, 8, 9, 10, 56, 52, 53, 54, 47, 49, 55, 51, 50, 48]
index = index[index.columns[new_order]]

In [32]:
index.head().transpose()

datetime,2007-01-01 00:00:00,2007-01-01 00:00:00,2007-01-01 00:00:00,2007-01-01 00:00:00,2007-01-01 10:00:00
index_name,Bank Holiday,Bank Holiday,Bank Holiday,Bank Holiday,Manufacturing PMI
impact,0,0,0,0,3
actual_stu,0,0,0,0,0
actual,NA,NA,NA,NA,54.8
forecast,NA,NA,NA,NA,NA
previous,NA,NA,NA,NA,55.3
num_actual,0,0,0,0,54.8
num_forecast,0,0,0,0,0
num_previous,0,0,0,0,55.3
bond_a_interest,0,0,0,0,0


In [33]:
index['index_name'].value_counts()

Bank Holiday                           1070
Unemployment Rate                      1065
Trade Balance                          1057
Retail Sales m/m                        770
Unemployment Claims                     626
Crude Oil Inventories                   626
Natural Gas Storage                     626
CB Leading Index m/m                    537
Current Account                         528
Manufacturing PMI                       525
Industrial Production m/m               432
CPI m/m                                 432
PPI m/m                                 432
Retail Sales y/y                        420
10-y Bond Auction                       386
Final Manufacturing PMI                 366
Core Retail Sales m/m                   338
PPI y/y                                 288
Consumer Confidence                     288
CPI y/y                                 288
Employment Change                       288
Housing Starts                          288
Core CPI m/m                    

In [35]:
index.columns

Index(['index_name', 'impact', 'actual_stu', 'actual', 'forecast', 'previous',
       'num_actual', 'num_forecast', 'num_previous', 'bond_a_interest',
       'bond_a_bid', 'bond_f_interest', 'bond_f_bid', 'bond_p_interest',
       'bond_p_bid', 'Status_Yes', 'Status_No', 'Status_Pass', 'Status_Reject',
       'Status_Leave', 'f_Status_Yes', 'f_Status_No', 'f_Status_Pass',
       'f_Status_Reject', 'f_Status_Leave', 'p_Status_Yes', 'p_Status_No',
       'p_Status_Pass', 'p_Status_Reject', 'p_Status_Leave', 'Votes_support',
       'Votes_oppose', 'Votes_neutral', 'f_Votes_support', 'f_Votes_oppose',
       'f_Votes_neutral', 'p_Votes_support', 'p_Votes_oppose',
       'p_Votes_neutral', 'year', 'month', 'day', 'weekday', 'time_Allday',
       'time_Event', 'time_month', 'time_Quarter', 'USD', 'EUR', 'GBP', 'JPY',
       'AUD', 'CAD', 'NZD', 'CNY', 'CHF', 'All'],
      dtype='object')

In [36]:
# 나머지 전처리
index = index.reset_index()
index['date'] = index['datetime'].apply(lambda x : x.strftime("%Y-%m-%d"))
index.set_index('date', inplace = True)
index.drop('datetime', axis=1, inplace=True)
index.drop(['index_name', 'actual', 'forecast', 'previous'], axis=1, inplace=True)
index = index.astype(float)

In [44]:
index.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53323 entries, 2007-01-01 to 2018-12-31
Data columns (total 57 columns):
impact             53323 non-null float64
actual_stu         53323 non-null float64
num_actual         53323 non-null float64
num_forecast       53323 non-null float64
num_previous       53323 non-null float64
bond_a_interest    53323 non-null float64
bond_a_bid         53323 non-null float64
bond_f_interest    53323 non-null float64
bond_f_bid         53323 non-null float64
bond_p_interest    53323 non-null float64
bond_p_bid         53323 non-null float64
Status_Yes         53323 non-null float64
Status_No          53323 non-null float64
Status_Pass        53323 non-null float64
Status_Reject      53323 non-null float64
Status_Leave       53323 non-null float64
f_Status_Yes       53323 non-null float64
f_Status_No        53323 non-null float64
f_Status_Pass      53323 non-null float64
f_Status_Reject    53323 non-null float64
f_Status_Leave     53323 non-null float64


* 변수 설명(actual : 실제, forecast : 예측, previous : 이전)
        impact             사건 중요도
        actual_stu         forecast->actual 변화가 경제에 미치는 영향(긍정:1, 부정:-1, 없음:0)
        num_actual         actual 수치
        num_forecast       forecast 수치
        num_previous       previous 수치
        bond_a_interest    채권 actual 이자율
        bond_a_bid         채권 actual 차익
        bond_f_interest    채권 forecast 이자율
        bond_f_bid         채권 forecast 차익
        bond_p_interest    채권 previous 이자율
        bond_p_bid         채권 previous 차익
        Status_Yes         actual 사건 상태
        Status_No          actual 사건 상태
        Status_Pass        actual 사건 상태
        Status_Reject      actual 사건 상태
        Status_Leave       actual 사건 상태
        f_Status_Yes       forecast 사건 상태
        f_Status_No        forecast 사건 상태
        f_Status_Pass      forecast 사건 상태
        f_Status_Reject    forecast 사건 상태
        f_Status_Leave     forecast 사건 상태
        p_Status_Yes       previous 사건 상태
        p_Status_No        previous 사건 상태
        p_Status_Pass      previous 사건 상태
        p_Status_Reject    previous 사건 상태
        p_Status_Leave     previous 사건 상태
        Votes_support      actual 투표 지지
        Votes_oppose       actual 투표 반대
        Votes_neutral      actual 투표 기권
        f_Votes_support    forecast 투표 지지
        f_Votes_oppose     forecast 투표 반대
        f_Votes_neutral    forecast 투표 기권
        p_Votes_support    previous 투표 지지
        p_Votes_oppose     previous 투표 반대
        p_Votes_neutral    previous 투표 기권
        year               연도
        month              월
        day                일
        weekday            요일
        time_Allday        시간 - 하루 전체
        time_Event         시간 - 행사일
        time_month         시간 - 월말 발표
        time_Quarter       시간 - 분기 발표
        USD                미국(달러)
        EUR                유럽(유로)
        GBP                영국(파운드)
        JPY                일본(앤화)
        AUD                호주(오스트레일리아 달러)
        CAD                캐나다(캐나다 달러)
        NZD                뉴질랜드(뉴질랜드 달러)
        CNY                중국(런민비)
        CHF                스위스(프랑)
        All                기타
        fa_num_rate        forecast -> actual 수치 변화율
        a_Votes_rate       actual 투표율
        f_Votes_rate       forecast 투표율
        fa_Votes_rate      forecast -> actual 투표율 변화율

In [41]:
# 각 종 긍정/부정 확인이 되는 변화율 데이터 변수 생성
index['fa_num_rate'] = np.abs(index['num_actual'] - index['num_forecast']) / index['num_forecast']
index.loc[index['fa_num_rate'] == np.inf, 'fa_num_rate'] = 1
index['fa_num_rate'] = index['fa_num_rate'] * index['actual_stu'] * index['impact']**2

index['a_Votes_rate'] = index['Votes_support'] / (index['Votes_support'] + index['Votes_oppose'] + index['Votes_neutral'])
index.loc[index['a_Votes_rate'] == np.inf, 'a_Votes_rate'] = 1
index['f_Votes_rate'] = index['f_Votes_support'] / (index['f_Votes_support'] + index['f_Votes_oppose'] + index['f_Votes_neutral'])
index.loc[index['f_Votes_rate'] == np.inf, 'f_Votes_rate'] = 1
index['fa_Votes_rate'] = np.abs(index['a_Votes_rate'] - index['f_Votes_rate']) / index['f_Votes_rate']
index.loc[index['fa_Votes_rate'] == np.inf, 'fa_Votes_rate'] = 1
index['fa_Votes_rate'] = index['fa_Votes_rate'] * index['actual_stu'] * index['impact']**2

In [45]:
# 새로운 데이터프레임 생성 및 저장(사건별 데이터 -> 일자별 데이터)
index_day = pd.DataFrame(index, columns=['fa_num_rate', 'fa_Votes_rate', 'time_Allday', 'time_Event', 'time_month', 'time_Quarter', 'USD', 'EUR', 'GBP', 'JPY', 'AUD', 'CAD', 'NZD', 'CNY', 'CHF', 'All'])
index_day = index_day.reset_index()
index_save = index_day.groupby('date').sum()
index_save.to_csv('data/index_merge.csv')

In [46]:
index_save.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3827 entries, 2007-01-01 to 2018-12-31
Data columns (total 16 columns):
fa_num_rate      3827 non-null float64
fa_Votes_rate    3827 non-null float64
time_Allday      3827 non-null float64
time_Event       3827 non-null float64
time_month       3827 non-null float64
time_Quarter     3827 non-null float64
USD              3827 non-null float64
EUR              3827 non-null float64
GBP              3827 non-null float64
JPY              3827 non-null float64
AUD              3827 non-null float64
CAD              3827 non-null float64
NZD              3827 non-null float64
CNY              3827 non-null float64
CHF              3827 non-null float64
All              3827 non-null float64
dtypes: float64(16)
memory usage: 508.3+ KB


* 변수 설명
        fa_num_rate : forecast -> actual 변화율
        fa_Votes_rate : forecast -> actual 투표율 변화율
        time_Allday : 하루 전체 시간에 해당되는 경우
        time_Event : 사건(행사)에 해당되는 경우
        time_month : 월별 발표에 해당되는 경우
        time_Quarter : 분기 발표에 해당되는 경우
        USD~All : 각 국가 및 화폐와 연관된 경우